## Установка пакета, необходимого для подключения к MsSQL
Пакет необходимо установить только один раз.  

При следующем запуске устанавливать не надо.

In [1]:
#Установка пакета
pip install pymssql

Note: you may need to restart the kernel to use updated packages.


## Импорт пакетов, необходимых для работы.
`ElementTree` - пакет, необходимый для работы с **XML**-файлами.  
`Pandas, numpy` - стандартные пакеты для анализа данных.  
`Pymssql` - пакет, для подключения к **MsSQL**.

In [1]:
#Импорт пакетов
from xml.etree import ElementTree
import pandas as pd
import numpy as np
import pymssql as ms

C:\Users\Alexey.Toporov\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """


## Подключение к базе данных.
В `iStar_XML` прописывается путь к файлу, в который будут выгружены данные. Если путь не прописывать, тогда файл создаётся в корневой папке.  

В `conn` указываются:
- `server` - имя сервера;
- `user` - имя пользователя;
- `password` - пароль пользователя;
- `database` - имя базы, откуда берутся данные.  

В `cursor.execute` необходимо указать промежуток времени для извлечения данных.  

In [76]:
#Подключение к MsSQL
iStar_XML = open('iStar_XML.txt', mode='w')

conn = ms.connect(server='SSQL', user='AEROPORT\Alexey.Toporov', 
                       password='', database='iStarRQ')
cursor = conn.cursor()
cursor.execute('''select cast(lg_ToTime as date), cast (lg_ReportStruct as xml) 
from iStarRQ.dbo.Log with (nolock) 
where lg_ReportStruct is not null and lg_ToTime between '2019-07-01' and '2020-01-15' ''')
row = cursor.fetchone()
while row:
    iStar_XML.write(f'{row[0]};:,{row[1]}\n')
    row = cursor.fetchone()

## Ветка фильтра.

В `file` необходимо прописать путь для файла из которого извлекаются данные. Путь должен совпадать с `iStar_XML` из прошлой ячейки.  

В `df1` и `df2` можно изменить имя колонок в таблице.  

В `pd.ExcelWriter` прописывается путь к папке, в которой будет сохранён отчёт.  

В `df1.to_excel` и `df2.to_excel` можно изменить название листов в отчёте **Excel**.  

In [80]:
#Фильтр

df=[]
dff=[]
file = 'iStar_XML.txt'
count=1
with open (file) as f:
    for l in f:
        line1,line=l.split(';:,')
        dom_xml=ElementTree.fromstring(line)
        type_of_report=dom_xml.find('general/type')
        report_name=dom_xml.find('general/title')
        filter_name=dom_xml.findall('query/node/items/node/items/')
        filter1=dom_xml.findall('query/node/items/node/items/node/items/item/title')
        for name in filter_name:
                filter2=name.findall('items/item/title')
                for f2 in filter2:
                    df.append([line1, count,type_of_report.text,report_name.text,name.find('title').text, f2.text])
        for name in filter_name:
                dff.append([line1, count,type_of_report.text,report_name.text,name.find('title').text])
        count+=1

df1=pd.DataFrame(data=np.array(df), columns=('Дата', 'Номер','Тип отчёта','Название отчёта','filter','filter_name'))
df2=pd.DataFrame(data=np.array(dff), columns=('Дата', 'Номер','Тип отчёта','Название отчёта','filter'))
df1['Тип отчёта']=df1['Тип отчёта'].map(lambda x: 'audience' if x=='0' else 'duplication' )
df2['Тип отчёта']=df2['Тип отчёта'].map(lambda x: 'audience' if x=='0' else 'duplication' )
p=df2.groupby(['filter']).size().reset_index(name='count')
p['share%']=p['count']/p['count'].sum()*100
p.sort_values('count',ascending=False, inplace=True)
p1=df2[['Дата', 'filter']].groupby(['filter','Дата']).size().unstack().fillna(0).reset_index()
with pd.ExcelWriter('filter2.xlsx') as writer:  
    df2.to_excel(writer, sheet_name='filter')
    df1.to_excel(writer, sheet_name='filter_name')
    p.to_excel(writer, sheet_name='filter_name_pivot')
    p1.to_excel(writer, sheet_name='filter_name_stats_by_date')

## Ветка структуры. Колонки.
В `file` необходимо прописать путь для файла из которого извлекаются данные. Путь должен совпадать с `iStar_XML` из 3 ячейки.  

В `c`, `c1`, `c2` и `c3` можно изменить имя колонок в таблице.  

В `pd.ExcelWriter` прописывается путь к папке, в которой будет сохранён отчёт.  

В `c.to_excel`, `c1.to_excel`, `c2.to_excel` и `c3.to_excel` можно изменить название листов в отчёте **Excel**.  

In [78]:
#Колонки
df=[]
df1=[]
df2=[]
df3=[]
df4=[]
file = 'iStar_XML.txt'
count=1
with open (file) as f:
    for l in f:
        line1,line=l.split(';:,')
        dom_xml=ElementTree.fromstring(line)
        type_of_report=dom_xml.find('general/type')
        report_name=dom_xml.findall('general/title')
        columns=dom_xml.findall('structure/columns/edges/edge/title')
        columns1=dom_xml.findall('structure/columns/edges/edge/items/statistic/title')
        columns2=dom_xml.findall('structure/columns/edges/edge/items/udb/structure/group/title')
        columns3=dom_xml.findall('structure/columns/edges/edge/items/attribute/title')
        for v in report_name:
            for i in columns:
                df1.append([line1,count,type_of_report.text, v.text,i.text])
        for v in report_name:
            for i in columns1:
                df2.append([line1,count,type_of_report.text, v.text,i.text])
        for v in report_name:    
            for i in columns2:
                df3.append([line1,count,type_of_report.text, v.text,i.text])
        for v in report_name:
            for i in columns3:
                df4.append([line1,count,type_of_report.text, v.text,i.text])
        count+=1
c=pd.DataFrame(data=np.array(df1),columns=['Дата','Номер','Тип отчёта','Название отчёта','edge_title'])
c1=pd.DataFrame(data=np.array(df2),columns=['Дата','Номер','Тип отчёта','Название отчёта','statistics'])
c2=pd.DataFrame(data=np.array(df3),columns=['Дата','Номер','Тип отчёта','Название отчёта','udb_title'])
c3=pd.DataFrame(data=np.array(df4),columns=['Дата','Номер','Тип отчёта','Название отчёта','attribute_title'])
c['Тип отчёта']=c['Тип отчёта'].map(lambda x: 'audience' if x=='0' else 'duplication' )
c1['Тип отчёта']=c1['Тип отчёта'].map(lambda x: 'audience' if x=='0' else 'duplication' )
c2['Тип отчёта']=c2['Тип отчёта'].map(lambda x: 'audience' if x=='0' else 'duplication' )
c3['Тип отчёта']=c3['Тип отчёта'].map(lambda x: 'audience' if x=='0' else 'duplication' )
p=c1.groupby(['statistics']).size().reset_index(name='count')
p['share%']=p['count']/p['count'].sum()*100
p.sort_values('count',ascending=False, inplace=True)
p1=c1[['Дата', 'statistics']].groupby(['statistics','Дата']).size().unstack().fillna(0).reset_index()
with pd.ExcelWriter('columns2.xlsx') as writer:  
    c.to_excel(writer, sheet_name='edge_title')
    c1.to_excel(writer, sheet_name='statistics')
    c2.to_excel(writer, sheet_name='udb_title')
    c3.to_excel(writer, sheet_name='attribute_title')
    p.to_excel(writer, sheet_name='attribute_title_pivot')
    p1.to_excel(writer, sheet_name='attribute_title_pivot_by_date')

## Ветка структуры. Строки.
В `file` необходимо прописать путь для файла из которого извлекаются данные. Путь должен совпадать с `iStar_XML` из 3 ячейки.  

В `c1`, `c2` и `c3` можно изменить имя колонок в таблице.  

В `pd.ExcelWriter` прописывается путь к папке, в которой будет сохранён отчёт.  

В `c1.to_excel`, `c2.to_excel` и `c3.to_excel` можно изменить название листов в отчёте **Excel**.  

In [10]:
#Строки

df=[]
df1=[]
df2=[]
df3=[]
file = 'iStar_XML.txt'
count=1
with open (file) as f:
    for line in f:
        dom_xml=ElementTree.fromstring(line)
        type_of_report=dom_xml.find('general/type')
        report_name=dom_xml.findall('general/title')
        rows=dom_xml.findall('structure/rows/edges/edge/title')
        rows2=dom_xml.findall('structure/rows/edges/edge/items/udb/structure/group/title')
        rows3=dom_xml.findall('structure/rows/edges/edge/items/attribute/title')
        for v in report_name:
            for i in rows:
                df1.append([count,type_of_report.text,v.text,i.text])
        for v in report_name:    
            for i in rows2:
                df2.append([count,type_of_report.text,v.text,i.text])
        for v in report_name:
            for i in rows3:
                df3.append([count,type_of_report.text,v.text,i.text])
        count+=1
                        
c1=pd.DataFrame(data=np.array(df1),columns=['Номер','Тип отчёта','Название отчёта','edge_title'])
c2=pd.DataFrame(data=np.array(df2),columns=['Номер','Тип отчёта','Название отчёта','udb_title'])
c3=pd.DataFrame(data=np.array(df3),columns=['Номер','Тип отчёта','Название отчёта','attribute_title'])
c1['Тип отчёта']=c1['Тип отчёта'].map(lambda x: 'audience' if x=='0' else 'duplication' )
c2['Тип отчёта']=c2['Тип отчёта'].map(lambda x: 'audience' if x=='0' else 'duplication' )
c3['Тип отчёта']=c3['Тип отчёта'].map(lambda x: 'audience' if x=='0' else 'duplication' )
with pd.ExcelWriter('rows2.xlsx') as writer:  
    c1.to_excel(writer, sheet_name='edge_title')
    c2.to_excel(writer, sheet_name='udb_title')
    c3.to_excel(writer, sheet_name='attribute_title')

## Ветка структуры. Слои.
В `file` необходимо прописать путь для файла из которого извлекаются данные. Путь должен совпадать с `iStar_XML` из 3 ячейки.  

В `df1` и `df2` можно изменить имя колонок в таблице.  

В `pd.ExcelWriter` прописывается путь к папке, в которой будет сохранён отчёт.  

В `df1.to_excel` и `df2.to_excel` можно изменить название листов в отчёте **Excel**.   

In [11]:
#Слои

df=[]
dff=[]
file = 'iStar_XML.txt'
count=1
with open (file) as f:
    for line in f:
        dom_xml=ElementTree.fromstring(line)
        type_of_report=dom_xml.find('general/type')
        report_name=dom_xml.find('general/title')
        planes=dom_xml.findall('structure/planes/edges/')
        for name in planes:
                filter2=name.findall('items/udb/structure/group/title')
                for f2 in filter2:
                    df.append([count,type_of_report.text,report_name.text,name.find('title').text, f2.text])
        for name in planes:
                dff.append([count,type_of_report.text,report_name.text,name.find('title').text])
        count+=1
                    
df1=pd.DataFrame(data=np.array(df), columns=('Номер','Тип отчёта','Название отчёта','planes','planes_name'))
df2=pd.DataFrame(data=np.array(dff), columns=('Номер','Тип отчёта','Название отчёта','planes'))
df1['Тип отчёта']=df1['Тип отчёта'].map(lambda x: 'audience' if x=='0' else 'duplication' )
df2['Тип отчёта']=df2['Тип отчёта'].map(lambda x: 'audience' if x=='0' else 'duplication' )
p=df1.groupby(['planes_name']).size().reset_index(name='count')
p['share%']=p['count']/p['count'].sum()*100
p.sort_values('count',ascending=False, inplace=True)
with pd.ExcelWriter('planes2.xlsx') as writer:  
    df2.to_excel(writer, sheet_name='planes')
    df1.to_excel(writer, sheet_name='planes_name')
    p.to_excel(writer, sheet_name='planes_name_pivot')

## Количество отчетов по их типу по дням недели.


In [79]:
#Колонки
df=[]
file = 'iStar_XML.txt'
count=1
with open (file) as f:
    for l in f:
        line1,line=l.split(';:,')
        dom_xml=ElementTree.fromstring(line)
        type_of_report=dom_xml.find('general/type')
        df.append([line1, type_of_report.text])
c=pd.DataFrame(data=np.array(df),columns=['Дата','Тип отчёта'])
c['Тип отчёта']=c['Тип отчёта'].map(lambda x: 'audience' if x=='0' else 'duplication' )
p=c[['Дата', 'Тип отчёта']].groupby(['Тип отчёта','Дата']).size().unstack().fillna(0).reset_index()
p
with pd.ExcelWriter('report_type.xlsx') as writer:  
    p.to_excel(writer, sheet_name='report_type_by_date')
